## Project #1
## Create a Neural Network In Python

# Mohamed Helmy ElMalkh 201900859
Nueral network with linear and activation layers (RELU AND SEGMOID)
BONUS is added as the code is generic and you can add unlimited number of layers forward and backward 

In [10]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from itertools import compress
from sklearn import metrics
from numpy import random
import math

This function to generate random number,but bounded

In [11]:
def kaiming_uniform(fan_in, fan_out, nonlinearity="relu", **kwargs):
    assert nonlinearity == "relu", "Only relu supported currently"
    bound = math.sqrt(2) * math.sqrt(3/fan_in)
    u = random.rand(fan_in,fan_out) * (2*bound) - bound
    return u

# **Layers**

**parent** 

Linear Layer 
- the Forward is multiply each feature by a certain weight 
-the backword 

In [12]:
class Layer: 
  def forward(self,x): 
      raise NotImplementedError()
  def backward(self,out_grad):
      raise NotImplementedError()
  def __call__(self, *args):
      return self.forward(*args)

**Linear Layer**

In [13]:
class Linear(Layer):
    def __init__(self, in_features, out_features, bias=True):
        self.in_features = in_features
        self.out_features = out_features       
        self.weight = kaiming_uniform(in_features,out_features)
    def forward(self, X):
        self.input = X
        output = np.dot(X,self.weight)
        return output
    def backward(self,segma,rate):
      delta = np.dot(np.array(self.input).T,np.array(segma))
      out_grad = np.dot(segma,(self.weight).T)
      self.weight -=  rate * delta
      return out_grad


**Segmoid layer**

In [14]:
def segmoid_fun(x):
  return 1/(1 + np.exp(-x)) 

class Segmoid(Layer):
  def forward(self, X):

    self.output = segmoid_fun(X)

    return self.output
  def backward(self,loss):
    return np.array(loss)*(1-np.array(self.output))*np.array(self.output)



**ReLU**

In [15]:
class ReLU(Layer):
  def forward(self, X):
    self.input = X
    return np.maximum(X,0)
  def backward(self,loss):
    return loss * (self.input > 0)

**Loss**

In [16]:
class Loss:
  def loss_grad(self,y,predict):
    error  = (np.sum((predict - y)**2,axis= 0)/y.shape[0]).mean()
    grad = -2*(y - predict)/len(y)
    return error,grad

## Test your model on IRIS  dataset and MNIST Dataset

# **IRIS TEST**

In [17]:
iris_raw = datasets.load_iris();
iris_df = pd.DataFrame(data = iris_raw.data , columns = iris_raw.feature_names)
input = iris_raw.data
output = pd.DataFrame(data = iris_raw.target , columns = ['target'])

X_train , X_test , y_train , y_test = train_test_split(output,iris_raw.target ,test_size = 0.2 , random_state=42 , shuffle = True)


scaler = StandardScaler()
input = scaler.fit_transform(X_train)
TestInput = scaler.fit_transform(X_test)


H = OneHotEncoder()
reshaped= y_train.reshape(len(y_train), 1)
y = H.fit_transform(reshaped).toarray()


reshapedO= y_test.reshape(len(y_test), 1)
TestOutput = H.fit_transform(reshapedO).toarray()

In [18]:
input_size = input.shape
input_size

(120, 1)

In [19]:
out_size = output.target.unique().shape
out_size[0]

3

In [20]:
lin_1 = Linear(input.shape[1],20)
act_1 = Segmoid()


lin_2 = Linear(20,10)
act_2 = Segmoid()

lin_3 = Linear(10,out_size[0])
act_3 = Segmoid()


loss = Loss()

for i in range(1000):
  out = act_3(lin_3(act_2(lin_2(act_1(lin_1(input))))))
  error,grad = loss.loss_grad(y,out)
  if(error <= 0):
    break

  lin_1.backward(act_1.backward(lin_2.backward(act_2.backward(lin_3.backward(act_3.backward(grad),0.1)),0.1)),0.1)
 
  
print("last error = ",error)
out = act_3(lin_3(act_2(lin_2(act_1(lin_1(input))))))

last error =  0.0656203260600635


In [21]:
predicted = out.argmax(axis=1) 

In [22]:
y_true = y.argmax(axis= 1)

In [23]:
(predicted == y_true).sum() /predicted.shape[0]

1.0

# Test the model

In [24]:
out = act_3(lin_3(act_2(lin_2(act_1(lin_1(TestInput))))))

In [25]:
predicted = out.argmax(axis=1) 

In [26]:
y_true = TestOutput.argmax(axis= 1)

In [27]:
(predicted == y_true).sum() /predicted.shape[0]

1.0

# **MINSIT TEST**

In [46]:
digits = datasets.load_digits()
digits_df = pd.DataFrame(data = digits['data'], columns = digits['feature_names'])
input = digits.data
output = pd.DataFrame(data = digits.target , columns = ['target'])

X_train , X_test , y_train , y_test = train_test_split(output,digits.target ,test_size = 0.2 , random_state=42 , shuffle = True)


scaler = StandardScaler()
input = scaler.fit_transform(X_train)
TestInput = scaler.fit_transform(X_test)


H = OneHotEncoder()
reshaped= y_train.reshape(len(y_train), 1)
y = H.fit_transform(reshaped).toarray()


reshapedO= y_test.reshape(len(y_test), 1)
TestOutput = H.fit_transform(reshapedO).toarray()

In [47]:
input_size = input.shape
input_size

(1437, 1)

In [48]:
out_size = output.target.unique().shape
out_size[0]

10

In [49]:
lin_1 = Linear(input.shape[1],30)
act_1 = ReLU()


lin_2 = Linear(30,19)
act_2 = Segmoid()

lin_3 = Linear(19,out_size[0])
act_3 = Segmoid()


loss = Loss()

for i in range(1000):
  out = act_3(lin_3(act_2(lin_2(act_1(lin_1(input))))))
  error,grad = loss.loss_grad(y,out)
  if(error <= 0):
    break

  lin_1.backward(act_1.backward(lin_2.backward(act_2.backward(lin_3.backward(act_3.backward(grad),0.3)),0.1)),0.5)
 
  
print("last error = ",error)
out = act_3(lin_3(act_2(lin_2(act_1(lin_1(input))))))

last error =  0.061695965620980196


In [50]:
predicted = out.argmax(axis=1) 

In [51]:
y_true = out.argmax(axis=1) 

In [52]:
(predicted == y_true).sum() /predicted.shape[0]

1.0

# Test the model

In [53]:
out = act_3(lin_3(act_2(lin_2(act_1(lin_1(TestInput))))))
predicted = out.argmax(axis=1) 
y_true = TestOutput.argmax(axis=1) 

In [54]:
(predicted == y_true).sum() /predicted.shape[0]

0.7472222222222222

## Refrence
https://becominghuman.ai/understanding-neural-networks-2-the-math-of-neural-networks-in-3-equations-6085fd3f09df
https://towardsdatascience.com/how-to-define-a-neural-network-as-a-mathematical-function-f7b820cde3f
